<a href="https://colab.research.google.com/github/soumik12345/enhance-me/blob/mirnet/notebooks/enhance_me_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/soumik12345/enhance-me -b mirnet
!pip install -qqq wandb streamlit

Cloning into 'enhance-me'...
remote: Enumerating objects: 89, done.
remote: Counting objects: 100% (89/89), done.
remote: Compressing objects: 100% (61/61), done.
remote: Total 89 (delta 43), reused 63 (delta 23), pack-reused 0
Unpacking objects: 100% (89/89), done.
     |████████████████████████████████| 1.7 MB 8.2 MB/s 
     |████████████████████████████████| 9.1 MB 33.4 MB/s 
     |████████████████████████████████| 140 kB 74.7 MB/s 
     |████████████████████████████████| 97 kB 8.6 MB/s 
     |████████████████████████████████| 180 kB 83.6 MB/s 
     |████████████████████████████████| 63 kB 2.1 MB/s 
     |████████████████████████████████| 4.3 MB 83.4 MB/s 
     |████████████████████████████████| 178 kB 68.0 MB/s 
     |████████████████████████████████| 76 kB 6.0 MB/s 
     |████████████████████████████████| 111 kB 81.8 MB/s 
     |████████████████████████████████| 125 kB 86.7 MB/s 
     |████████████████████████████████| 791 kB 67.2 MB/s 
     |████████████████████████████████| 374 

In [2]:
import sys
sys.path.append("./enhance-me")

from PIL import Image
from enhance_me import commons
from enhance_me.mirnet import MIRNet

In [3]:
#@title MIRNet Train Configs

experiment_name = 'lol_dataset_256' #@param {type:"string"}
image_size = 128 #@param {type:"integer"}
dataset_label = 'lol' #@param ["lol"]
apply_random_horizontal_flip = True #@param {type:"boolean"}
apply_random_vertical_flip = True #@param {type:"boolean"}
apply_random_rotation = True #@param {type:"boolean"}
wandb_api_key = '' #@param {type:"string"}
val_split = 0.1 #@param {type:"slider", min:0.1, max:1.0, step:0.1}
batch_size = 4 #@param {type:"integer"}
num_recursive_residual_groups = 3 #@param {type:"slider", min:1, max:5, step:1}
num_multi_scale_residual_blocks = 2 #@param {type:"slider", min:1, max:5, step:1}
learning_rate = 1e-4 #@param {type:"number"}
epsilon = 1e-3 #@param {type:"number"}
epochs = 50 #@param {type:"slider", min:10, max:100, step:5}

In [4]:
mirnet = MIRNet(
    experiment_name=experiment_name,
    wandb_api_key=None if wandb_api_key == '' else wandb_api_key
)

wandb: Currently logged in as: 19soumik-rakshit96 (use `wandb login --relogin` to force relogin)


In [5]:
mirnet.build_datasets(
    image_size=image_size,
    dataset_label=dataset_label,
    apply_random_horizontal_flip=apply_random_horizontal_flip,
    apply_random_vertical_flip=apply_random_vertical_flip,
    apply_random_rotation=apply_random_rotation,
    val_split=val_split,
    batch_size=batch_size
)

347185152/347171015 [==============================] - 13s 0us/step
Number of train data points: 436
Number of validation data points: 49


In [6]:
mirnet.build_model(
    num_recursive_residual_groups=num_recursive_residual_groups,
    num_multi_scale_residual_blocks=num_multi_scale_residual_blocks,
    learning_rate=learning_rate,
    epsilon=epsilon
)

In [ ]:
history = mirnet.train(epochs=epochs)

/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


Epoch 1/50
 66/218 [========>.....................] - ETA: 2:25 - loss: 0.1721 - peak_signal_noise_ratio: 63.2555

In [ ]:
for index, low_image_file in enumerate(mirnet.test_low_images):
    original_image = Image.open(low_image_file)
    enhanced_image = mirnet.infer(original_image)
    ground_truth = Image.open(mirnet.test_enhanced_images[index])
    commons.plot_results(
        [original_image, ground_truth, ground_truth],
        ["Original Image", "Ground Truth", "Enhanced Image"],
        (18, 18)
    )